In [4]:
import numpy as np
import tensorflow as tf

from tensorflow import convert_to_tensor, string
from tensorflow.keras.layers import Embedding, Layer, LayerNormalization, Dense, ReLU, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow import matmul, reshape, shape, transpose, cast, float32
from keras.backend import softmax

In [ ]:

class PositionEmbeddingFixedWeights(Layer):
    def __init__(self, sequence_length, vocab_size, output_dim, **kwargs):
        super(PositionEmbeddingFixedWeights, self).__init__(**kwargs)
        word_embedding_matrix = self.get_position_encoding(vocab_size, output_dim)   
        position_embedding_matrix = self.get_position_encoding(sequence_length, output_dim)                                          
        self.word_embedding_layer = Embedding(
            input_dim=vocab_size, output_dim=output_dim,
            weights=[word_embedding_matrix],
            trainable=False
        )
        self.position_embedding_layer = Embedding(
            input_dim=sequence_length, output_dim=output_dim,
            weights=[position_embedding_matrix],
            trainable=False
        )
             
    def get_position_encoding(self, seq_len, d, n=10000):
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return P
 
 
    def call(self, inputs):        
        position_indices = tf.range(tf.shape(inputs)[-1])
        embedded_words = self.word_embedding_layer(inputs)
        embedded_indices = self.position_embedding_layer(position_indices)
        return embedded_words + embedded_indices

In [5]:
class DotProductAttention(Layer):
    def __init__(self, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)

    def call(self, queries, keys, values, d_k, mask=None):
        scores = matmul(queries, keys, transpose_b=True) / tf.math.sqrt(cast(d_k, float32))
        
        if mask is not None:
            score += -1e9 * mask

        weight = softmax(scores)

        return matmul(weight, values)

class MultiHeadAttention(Layer):
    def __init__(self, h, d_k, d_v, d_model, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.attention = DotProductAttention()
        self.head = h
        self.d_k = d_k
        self.d_v = d_v
        self.d_model = d_model
        self.W_q = Dense(d_k)
        self.W_k = Dense(d_k)
        self.W_v = Dense(d_v)
        self.W_o = Dense(d_model)

    def reshape_tensor(self, x, head, flag):
        if flag:
            x = reshape(x, shape=(shape(x)[0], shape(x)[1], head, -1))
            x = transpose(x, perm=(0, 2, 1, 3))
        else:
            x = transpose(x, perm=(0, 2, 1, 3))
            x = reshape(x, shape=(shape(x)[0], shape(x)[1], self.d_k))
        return x

    def call(self, queries, keys, values, mask=None):
        q_reshaped = self.reshape_tensor(self.W_q(queries), self.head, True)
        k_reshaped = self.reshape_tensor(self.W_k(keys), self.head, True)
        v_reshaped = self.reshape_tensor(self.W_v(values), self.head, True)

        o_reshaped = self.attention(q_reshaped, k_reshaped, v_reshaped, self.d_k, mask)
        
        output = self.reshape_tensor(o_reshaped, self.head, False)

        return self.W_o(output)

### Implementing the Add & Norm Layer

In [7]:
class AddNormalization(Layer):
    def __init__(self, **kwargs):
        super(AddNormalization, self).__init__(**kwargs)
        self.layer_norm = LayerNormalization()

    def call(self, x, sublayer_x):
        add = x + sublayer_x
        return self.layer_norm(add)

### Implementing the Feed-Forward Layer

In [19]:
class FeedForward(Layer):
    def __init__(self, d_ff, d_model, **kwargs):
        super(FeedForward, self).__init__(**kwargs)
        self.fully_connected1 = Dense(d_ff)
        self.fully_connected2 = Dense(d_model)
        self.activation = ReLU()

    def call(self, x):
        x_fc1 = self.fully_connected1(x)
        return self.fully_connected2(self.activation(x_fc1))

### Implementing the Encoder Layer

In [28]:
class EncoderLayer(Layer):
    def __init__(self, h, d_k, d_v, d_model, d_ff, rate, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.muti_head_attention = MultiHeadAttention(h, d_k, d_v, d_model)
        self.dropout1 = Dropout(rate)
        self.add_norm1 = AddNormalization()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout2 = Dropout(rate)
        self.add_norm2 = AddNormalization()

    def call(self, x, padding_mask, training):
        multihead_output = self.muti_head_attention(x, x, x, padding_mask)
        multihead_output = self.dropout1(multihead_output, training=training)
        addnorm_output = self.add_norm1(x, multihead_output)
        feedforward_output = self.feed_forward(addnorm_output)
        feedforward_output = self.dropout2(feedforward_output, training=training)
        return self.add_norm2(addnorm_output, feedforward_output)

### Implementing the Encoder

In [24]:
class Encoder(Layer):
    def __init__(self, vocab_size, sequence_length, h, d_k, d_v, d_model, d_ff, n, rate, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.pos_encoding = PositionEmbeddingFixedWeights(sequence_length, vocab_size, d_model)
        self.dropout = Dropout(rate)
        self.encoder_layer = [EncoderLayer(h, d_k, d_v, d_model, d_ff, rate) for _ in range(n)]

    def call(self, input_sentence, padding_mask, training):
        pos_encoding_output = self.pos_encoding(input_sentence)
        x = self.dropout(pos_encoding_output, training=training)

        for i, layer in enumerate(self.encoder_layer):
            x = layer(x, padding_mask, training)

        return x

### Test

In [29]:
from numpy import random

enc_vocab_size = 20 # Vocabulary size for the encoder
input_seq_length = 5  # Maximum length of the input sequence
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected queries and keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 2048  # Dimensionality of the inner fully connected layer
d_model = 512  # Dimensionality of the model sub-layers' outputs
n = 6  # Number of layers in the encoder stack
 
batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in the dropout layers
 
input_seq = random.random((batch_size, input_seq_length))
 
encoder = Encoder(enc_vocab_size, input_seq_length, h, d_k, d_v, d_model, d_ff, n, dropout_rate)
print(encoder(input_seq, None, True))

tf.Tensor(
[[[-1.45917624e-01 -9.15344059e-01  1.45258874e-01 ...  2.06214339e-01
   -1.76602054e+00  2.61561185e-01]
  [ 3.41569073e-02 -6.05697691e-01 -4.38042223e-01 ...  3.00020933e-01
   -1.73444092e+00  2.31028005e-01]
  [ 4.25433069e-01 -1.21905255e+00  2.99051464e-01 ... -2.81086773e-01
   -1.62082088e+00  3.20402622e-01]
  [ 1.99012801e-01 -1.45455492e+00  3.18659157e-01 ...  1.19393110e-01
   -1.69570315e+00  2.39699095e-01]
  [-8.49100649e-02 -1.70686495e+00  3.44290167e-01 ...  1.34133554e+00
   -1.74305832e+00 -3.48057628e-01]]

 [[ 6.91816151e-01 -4.11242127e-01  7.75505662e-01 ...  6.14771128e-01
   -1.29956734e+00 -2.73143202e-01]
  [ 3.14976186e-01  2.32824728e-01  1.57901540e-01 ... -1.40927851e-01
   -6.43691003e-01 -1.75628498e-01]
  [ 8.16481352e-01 -1.51634872e+00  4.83508129e-03 ...  7.69579351e-01
   -6.14210725e-01  8.06200653e-02]
  [ 4.98605371e-01 -7.62940168e-01 -9.90712196e-02 ...  1.08738191e-01
   -6.31547511e-01  3.07440966e-01]
  [-3.76309529e-02 -3.87

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b5f739dc-f641-4c72-a448-d84edd2bf5bd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>